In [52]:
import pandas as pd
import datetime as dt

In [53]:
marriages = pd.read_csv("../../data/Base-Matrimonios-2000-al-31_03_2015seg.csv")
divorces = pd.read_csv("../../data/bases-divorcios-2000-31_03_2015-csv.csv")

In [54]:
# Creating marriage df with only marriage date, both DOB's, and an empty flag for divorce
marriages = marriages[["Fecha","ElFecNac","EllaFecNac"]]
marriages = marriages.rename(columns={"Fecha":"Marriage_Date","ElFecNac":"DOB1","EllaFecNac":"DOB2"})
marriages[["Marriage_Date","Marriage_Time"]] = marriages["Marriage_Date"].str.split("T",expand=True)
marriages[["DOB1","DOB1_Time"]] = marriages["DOB1"].str.split("T",expand=True)
marriages[["DOB2","DOB2_Time"]] = marriages["DOB2"].str.split("T",expand=True)
marriages = marriages[["Marriage_Date","DOB1","DOB2"]]
marriages.head()

,Marriage_Date,DOB1,DOB2
0,2000-01-03,1974-08-20,1980-01-01
1,2000-01-05,1980-02-12,1981-12-12
2,2000-01-05,1978-11-26,1979-09-04
3,2000-01-06,1974-10-30,1978-12-04
4,2000-01-06,1937-03-13,1956-09-11


In [55]:
# Creating divorce df with only marriage date, and both DOB's
divorces = divorces[["Fecha","F_Naci_Do","F_Naci_Da"]]
divorces = divorces.rename(columns={"Fecha":"Marriage_Date","F_Naci_Do":"DOB1","F_Naci_Da":"DOB2"})
divorces = divorces.dropna()
divorces['Marriage_Date'] = pd.to_datetime(divorces['Marriage_Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
divorces['DOB1'] = pd.to_datetime(divorces['DOB1'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
divorces['DOB2'] = pd.to_datetime(divorces['DOB2'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
divorces["divorce"] = 1
divorces.head()

,Marriage_Date,DOB1,DOB2,divorce
0,2000-06-26,1975-12-18,1983-01-08,1
2,1975-12-18,1955-02-22,1947-03-21,1
4,1998-11-14,1975-10-30,1978-10-13,1
5,1995-01-20,1973-03-28,1976-06-14,1
6,1991-08-16,1970-12-13,1971-11-04,1


In [59]:

df = pd.merge(marriages, divorces, how='left', on=["Marriage_Date","DOB1","DOB2"])
df["divorce"] = df["divorce"].fillna(0)

In [60]:
df["divorce"].value_counts()

0.0    23465
1.0     1284
Name: divorce, dtype: int64